In [58]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go
import time
import pickle

# TODO estimate f(iteration) = len(_points)(iteration)
# TODO eliminate effects of the particular distribution while iteration <~ 5
# TODO f(iterations) plot (with deviations)

In [30]:
class Box(object):
  """Box with dots, the problem is to find their minimal convex hull"""
  def __init__(self, dimension=2, size=1, distribution='Uniform', points_number=100):
    self.dimension = dimension
    self.size = size
    self.distribution = distribution
    self.points_number = points_number
    self.convex_hull = []
    
    if distribution=='Uniform':
        points_array = np.array([
                      [random.random() for x_iterator in range(self.points_number)], 
                      [random.random() for y_iterator in range(self.points_number)]
                      ]).reshape((self.points_number,2))
    elif distribution=='Gauss':
        points_array = np.array([
                      [random.gauss(mu=1, sigma=1) for x_iterator in range(self.points_number)], 
                      [random.gauss(mu=1, sigma=1) for y_iterator in range(self.points_number)]
                      ]).reshape((self.points_number,2))
    self._points = pd.DataFrame(data=points_array, columns=("x", "y"))
    self.picture_data = px.scatter(self._points, x="x", y="y").data
                   

  def find_convex(self):
    def metric(vectors_array, vector):
        """Array of normalized dot products of each vec1 in vectors_array with single vector"""
        nvector = vector / np.linalg.norm(vector)
        nvectors_array = vectors_array / np.linalg.norm(vectors_array, axis=0)
        nvectors_array = np.column_stack((nvectors_array[0], nvectors_array[1]))
        return np.tensordot(nvectors_array, nvector, axes=1)
    
    data = self._points
    # The first point in the convex_hull is the lowest one
    lowest_point_index = (data.idxmin(axis=0).y)
    # print("_______________________________            lowest_point is ", lowest_point_index)
    self.convex_hull = [lowest_point_index]
    current_vector = np.array([1, 0])
    current_point_index = lowest_point_index
    looped_to_lowest = False
    # Then we go around points box counterclockwise: 
    while (looped_to_lowest == False):   
        difference_vectors = np.array([
            data.x - data.x[current_point_index],
            data.y - data.y[current_point_index]
        ])
        # Difference vector contains (current_point - current_point) = 0. Let's delete it:
        probe_vectors = np.delete(difference_vectors, (current_point_index), axis=1)
        metrics = metric(probe_vectors, current_vector)
        # Manually incert (current_point - current_point) metric to be the smallest and impossible: -2
        metrics = np.insert(metrics, current_point_index, -2)
        # print(len(metrics))
        next_point_index = np.where(metrics == np.amax(metrics))[0][0]
        # print(next_point_index)
        
        self.convex_hull.append(next_point_index)
        current_vector = np.array([
            data.x[next_point_index] - data.x[current_point_index],
            data.y[next_point_index] - data.y[current_point_index]
        ])
        current_point_index = next_point_index
        if current_point_index == lowest_point_index:
          looped_to_lowest = True
            
  def collect_convex(self, show=False):
    """Adds a new convex hull line to the figure data"""
    # TODO colormap for layers
    if len(self.convex_hull):
      convex_hull_df = self._points.loc[self._points.index[self.convex_hull]]
    else:
        convex_hull_df = pd.DataFrame(data=[], columns=("x", "y"))
        
    convex_hull_figure = px.line(convex_hull_df, x="x", y="y" )
    self.picture_data += convex_hull_figure.data
    if show:
        fig = go.Figure(data = self.picture_data)
        fig.show()
    
  def drop_convex(self):
    """Remove convex_hull from _points DataFrame"""
    self._points.drop(self.convex_hull, inplace=True, axis=0)
    self._points.reset_index(drop=True, inplace=True)
    self.convex_hull = []
      

In [62]:
start_time = time.time()

# points_set = [10, 100, 1000, 10000, 100000]
points_set = [100000, 10000, 1000, 100, 10]
for points_number in points_set:
    print("Number of points is %d" % points_number)
          
    hull_list = [] # list of hull sizes for all generations
    iteration_list = [] # how many hulls did it take to "eat" the whole set of points
    for generation in range(5):
        start_gener_time = time.time()
        
        box = Box(dimension=2, size=1, distribution='Uniform', points_number=points_number)
        hull_size = []
        
        iteration = 0
        while box._points.shape[0]:
            # Time for each iteration is less than a second even for points_number = 100000
            # Note that # of iterations is ~1000 in case of points_number = 100000
            # start_iter_time = time.time()
            box.find_convex()
            
            # Collect all convex hulls to one plotly object and to show it later. Takes time!
            # box.collect_convex(show=False)
            
            # Write down size of a particular hull (# of iteration is the position of a hull)
            hull_size.append(len(box.convex_hull))
            
            box.drop_convex()
            iteration += 1
            # print("--- Iteration %s, %s seconds ---" % (iteration, (time.time() - start_iter_time)))


        hull_list.append(hull_size)
        iteration_list.append(iteration)

        print("--- Generation %s, %s seconds ---" % (generation, (time.time() - start_gener_time)))


    iterations = np.array(iteration_list)
    
    filename = str(points_number) + '_points_hull'
    with open(filename, 'wb') as fp:
        pickle.dump(hull_list, fp)
    
    filename = str(points_number) + '_iterations'
    with open(filename, 'wb') as fp:
        pickle.dump(iteration_list, fp)
    
    print("Average iterations = ", np.average(iterations))
    print("Standard deviation = ", np.std(iterations))
    print("\n")

print("--- ", time.time() - start_time, " ---")

Number of points is 100000
--- Generation 0, 617.0175466537476 seconds ---
--- Generation 1, 588.6121821403503 seconds ---
--- Generation 2, 584.8050692081451 seconds ---
--- Generation 3, 627.5616917610168 seconds ---
--- Generation 4, 583.165194272995 seconds ---
Average iterations =  1047.8
Standard deviation =  2.6381811916545836


Number of points is 10000
--- Generation 0, 15.664625406265259 seconds ---
--- Generation 1, 15.451514482498169 seconds ---
--- Generation 2, 15.447160959243774 seconds ---
--- Generation 3, 15.63624906539917 seconds ---
--- Generation 4, 15.649001598358154 seconds ---
Average iterations =  225.6
Standard deviation =  0.8


Number of points is 1000
--- Generation 0, 1.3644187450408936 seconds ---
--- Generation 1, 1.3521130084991455 seconds ---
--- Generation 2, 1.3332080841064453 seconds ---
--- Generation 3, 1.3380115032196045 seconds ---
--- Generation 4, 1.3616671562194824 seconds ---
Average iterations =  49.6
Standard deviation =  0.8


Number of p

In [72]:
df = pd.DataFrame(hull_list)
df = df.transpose()
df = pd.melt(df, ignore_index=False, var_name="Experiment", value_name="Size")
df['Iteration'] = df.index
df.reset_index(inplace=True)
df.dropna(inplace=True)
df.head()
chart = px.line(df, x='Iteration', y='Size', color='Experiment')
chart.show()

In [71]:

with open('100000_points_hull', 'rb') as f:
     hull_list = pickle.load(f) 

In [84]:
iterations_df = pd.DataFrame([], columns = ['0','1','2','3','4'])
for points_number in points_set:
    filename = str(points_number) + '_iterations'
    with open(filename, 'rb') as f:
        iterations_df.loc[len(iterations_df)] = pickle.load(f)

In [114]:
df = iterations_df.iloc[0:5,1:2]
df = df.assign(x=points_set)

fig = px.line(df, x='x', y='1', log_x=True, log_y=True)
fig.show()